In [2]:
import glob
import numpy as np
# hide numpy log warning
np.seterr(divide = 'ignore') 
import pandas as pd
import warnings
import unittest

from biom import load_table, Table
from biom.util import biom_open
from scipy.signal import medfilt
from scipy.special import softmax
from skbio.stats.composition import closure
from numpy.random import poisson, lognormal, normal, randint
from tqdm.notebook import tqdm

# hide pandas Future/Deprecation Warning(s) for tutorial
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
def softmax_inv(x):
    return np.log(x)

class TestSoftInv(unittest.TestCase):

    def setUp(self):
        self.exp = closure(np.array([[100, 5, 20],
                                     [200, 234, 14]]))
        pass

    def testinv(self):

        res = softmax(softmax_inv(self.exp))
        res *= 1 / res.sum(axis=1)[0]
        try:
            np.testing.assert_array_almost_equal(res, self.exp, 6)
        except AssertionError:
            print('arrays not equal')

def simulate(X, depth=100, read_std=10.0,
             pnormal=0.1, pheter=0.1,
             pzero=0.1, mwindow=False):
    """
    Poisson Log-Normal simulation
    on real count data. The proceedure
    follows the same steps as
    Äijö et al. [1] but with a Poisson
    Log-Normal count model. This allows
    the total read depth to vary across
    samples which can be modeled by the
    dispersion perameter of the Poisson.

    Parameters
    ----------
    table : array like
        A matrix of counts.
    depth : int
        Mean read depth of the
        simulation.
    read_std : float
        Controls dispersion.
    pnormal : float [0,1]
        Percent
        normally-distributed
        noise to the data.
    pheter : float [0,1]
        Percent random
        noise to the data.
    pzero : float [0,1]
        Percent zeros added.

    Returns
    -------
    sim : array-like
        simulated counts.

    References
    ----------
    .. [1] Äijö, Tarmo, Christian L. Müller,
           and Richard Bonneau. 2018.
           “Temporal Probabilistic Modeling of Bacterial
           Compositions Derived from 16S rRNA Sequencing.”
           Bioinformatics  34 (3): 372–80.
    """

    # subset and order single digester
    sim = X.copy()
    # (1) estimate proportions
    sim = closure(sim)
    # (2) filter low-mean reads
    abund_filt_mask = sim.mean(axis=0) > 1e-5
    mask_index = [i for i, v in enumerate(abund_filt_mask) if v]
    sim = sim.T[abund_filt_mask].T
    # (3) median filter by feature
    if mwindow:
        for i in range(sim.shape[1]):
            sim[:, i] = medfilt(sim[:, i], mwindow)
    sim = closure(sim)  # check closure
    # (4) project onto real space
    # with inverse softmax
    sim = softmax_inv(sim)
    # (5) noise gauss (@10% of samps.)
    err = pnormal * np.ones_like(sim)
    sim = normal(sim, err)
    # add some random noise (@10% of samps.)
    err = pheter * np.ones_like(sim)
    i = randint(0, err.shape[0], 5000)
    j = randint(0, err.shape[1], 5000)
    err[i, j] = 0.5
    sim = normal(sim, err)
    # add random zeros (@10% of samps.)
    err = pzero * np.ones_like(sim)
    i = randint(0, err.shape[0], 100)
    j = randint(0, err.shape[1], 100)
    err[i, j] = 0.0
    sim = normal(sim, err)
    # (6) back to prop
    sim = closure(softmax(sim))
    # (7) Poisson - Log-Normal Counts
    mean = depth
    stdev = read_std
    phi = (stdev ** 2 + mean ** 2) ** 0.5
    mu = mean**2 / phi
    mu = np.log(mu * sim.T)
    sigma = (np.log(phi ** 2 / mean ** 2)) ** 0.5
    sim = np.vstack([poisson(lognormal(mu[:, i],
                                       sigma))
                     for i in range(sim.shape[0])])
    return sim, mask_index, sigma

def simulation_helper(bt, mf, 
                      subject_id_column,
                      state_column,
                      depth=100, read_std=10.0,
                      pnormal=0.1, pheter=0.1,
                      pzero=0.1, mwindow=False):
    """
    helper function to run simulation multiple times.
    """
    # full simulation table
    simulation_table = []
    # generate simulation by series
    table_df = pd.DataFrame(bt.matrix_data.toarray(), bt.ids('observation'), bt.ids())
    for subject_id, subject_mf in mf.groupby(subject_id_column):
        # subset to single series
        subject_mf = subject_mf.sort_values(state_column)
        table_df_ = table_df.copy().loc[:, subject_mf.index]
        table_df_ = table_df_[table_df_.sum(1) > 0].T
        # continue
        if table_df_.shape[0] <= 1:
            continue
        # generate simulation on series 
        # This is where you change data characteristics!!
        sim, mask_index, sigma = simulate(table_df_.values,
                                          depth=depth,
                                          read_std=read_std,
                                          pnormal=pnormal,
                                          pheter=pheter,
                                          pzero=pzero,
                                          mwindow=mwindow)
        # make dataframe
        simdf = pd.DataFrame(sim.T,
                             table_df_.columns[mask_index],
                             table_df_.index)
        simulation_table.append(simdf)
    # merge
    simulation_table = pd.concat(simulation_table, axis=1, sort=False).fillna(0)
    # convert table to biom
    simulation_table = Table(simulation_table.values, 
                             simulation_table.index, 
                             simulation_table.columns)
    return simulation_table

In [4]:
# load filtered tables
body_site = 'Baby-Feces'
mf = pd.read_csv('../data/%s/metadata-filtered.tsv' % body_site, index_col=0)
bt = load_table('../data/%s/table-filtered.biom' % body_site)

In [5]:
#make sure there are at least two time points per subject
subject_id_column = 'subjectid_unique'
ids_to_keep = mf[subject_id_column].value_counts()[mf[subject_id_column].value_counts() > 1].index
mf = mf[mf[subject_id_column].isin(ids_to_keep)]

In [ ]:
# inits
experiment = 'seq-depth'
results_path = '../results/{}-tables/'.format(experiment)
state_column =  'date_sampling_category_days_continuous'
simulations = {}
# example: altering the depth example
for exp in tqdm([500, 1000, 5000, 10000]):
    for fold in range(3):
        dispersion = int(exp/10)
        print('Depth: {}, fold: {}'.format(exp, fold))
        simulations[(dispersion, fold)] = simulation_helper(bt, mf, subject_id_column, state_column,
                                                            depth=exp, read_std=dispersion)
        
        with biom_open('{}table-{}-{}.biom'.format(results_path, exp, fold), 'w') as f:  
            simulations[(dispersion, fold)].to_hdf5(f, "{}-{}".format(dispersion, fold))

In [8]:
# inits
experiment = 'dispersion'
results_path = '../results/{}-tables/'.format(experiment)
state_column =  'date_sampling_category_days_continuous'
simulations = {}
# example: altering the depth example
for exp in tqdm([5,10,20,50]):
    for fold in range(3):
        seq_depth = 1000
        dispersion = int(seq_depth/exp)
        print('Dispersion: {}, fold: {}'.format(seq_depth/exp, fold))
        simulations[(dispersion, fold)] = simulation_helper(bt, mf, subject_id_column, state_column,
                                                            depth=seq_depth, read_std=dispersion)
        
        with biom_open('{}table-{}-{}-{}.biom'.format(results_path, seq_depth, dispersion, fold), 'w') as f:  
            simulations[(dispersion, fold)].to_hdf5(f, "{}-{}".format(dispersion, fold))

  0%|          | 0/4 [00:00<?, ?it/s]

Dispersion: 200.0, fold: 0
Dispersion: 200.0, fold: 1
Dispersion: 200.0, fold: 2
Dispersion: 100.0, fold: 0
Dispersion: 100.0, fold: 1
Dispersion: 100.0, fold: 2
Dispersion: 50.0, fold: 0
Dispersion: 50.0, fold: 1
Dispersion: 50.0, fold: 2
Dispersion: 20.0, fold: 0
Dispersion: 20.0, fold: 1
Dispersion: 20.0, fold: 2
